# Basketball-Reference Game Log Data Scraper
_To run any of the sections of code, click into the section, and press [Ctrl]+[Enter]_  
__Note: In order for this code to function correctly, it must be executed chronologically, starting at the top__

## Install/Import required Python Libraries

In [1]:
#If necessary, install BeautifulSoup4 and Pandas Packages:
!pip install requests
!pip install BeautifulSoup4
!pip install pandas
!pip install tqdm
!pip install numpy

You should consider upgrading via the 'c:\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python37\python.exe -m pip install --upgrade pip' command.


First use Kernel [3] to import all of the required python libraries for this project.  
***
If any library does not load, uncomment (remove the # before) the associated '!pip install' command in Kernel [1]

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import unicodedata
import numpy
from tqdm import tqdm
from time import sleep

## Select the years/months that you want to pull game data from:

In [3]:
#Select the year you want to pull data from - only limited to single years for the time being.
start_year = input('----Input the Start Year:  ')
stop_year = input('----Input the Final Year:  ')
years = range(int(start_year),int(stop_year)+1)

#Comment out any months you DO NOT want game data pulled from:
months = [
'october', 
'november',
'december',
'january',
'february',
'march',
'april',
'may',
'june',
'july',
'august',
'september'
]

----Input the Start Year:   2021
----Input the Final Year:   2021


In [4]:
def make_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    return soup

In [5]:
links = []
links_tmp = []
with tqdm(total=len(months)*len(years),desc='Link Creation Progress') as pbar:
    for year in years:
        for month in months:
            soup = make_soup('https://www.basketball-reference.com/leagues/NBA_'+str(year)+'_games-'+month+'.html')
            links_tmp = ["https://www.basketball-reference.com" + game['href'] for game in soup.find_all('a', text='Box Score')]
            for link in links_tmp:
                links.append(link)
            pbar.update(1)

print('Links successfully created')

Link Creation Progress: 100%|██████████████████████████████████████████████████████████| 12/12 [00:03<00:00,  3.10it/s]

Links successfully created


## Scraping the Box Score Data:

In [15]:
stats = []
with tqdm(total=len(links),desc='Scraping Progress') as pbar:
    for link in range(len(links)):
    #for link in range():
        tmp = links[link]
        page = requests.get(tmp)
        soup = BeautifulSoup(page.text, 'html.parser')

        teams = [game['href'] for game in soup.find_all('a', {'itemprop': 'name'})]
        teams = [teams[0][7:10], teams[1][7:10]]

        stats_tmp = []
        for team in range(len(teams)):
            basic_stats = soup.find('table',{'id': 'box-'+teams[team]+'-game-basic'})
            adv_stats = soup.find('table',{'id': 'box-'+teams[team]+'-game-advanced'})

            #Player Names
            player_names = [[th.getText() for th in basic_stats.findAll('tr')[1:][i].findAll('th')] for i in range(len(basic_stats.findAll('tr')[1:]))]
            player_names = player_names[1:6] + player_names[7:-1]
            df_player_names = pd.DataFrame(player_names)

            #Starters
            starters = ['Y','Y','Y','Y','Y'] + ['N']*(len(player_names)-5)
            df_starters = pd.DataFrame(starters)
            
            #Player Basic Stats
            team_stats_basic = [[td.getText() for td in basic_stats.findAll('tr')[1:][i].findAll('td')]
                    for i in range(len(basic_stats.findAll('tr')[1:]))]
            team_stats_basic = team_stats_basic[1:6] + team_stats_basic[7:-1]
            df_team_stats_basic = pd.DataFrame(team_stats_basic)

            #Player Advanced Stats
            team_stats_adv = [[td.getText() for td in adv_stats.findAll('tr')[1:][i].findAll('td')]
                    for i in range(len(adv_stats.findAll('tr')[1:]))]
            team_stats_adv = team_stats_adv[1:6] + team_stats_adv[7:-1]
            df_team_stats_adv = pd.DataFrame(team_stats_adv)

            #Game ID
            game_id = links[link][47:59]
            game_id = [game_id for i in range(len(player_names))]
            df_game_id = pd.DataFrame(game_id)
            
            #Opponent ID
            if team == 0:
                opponent = teams[1]
            else:
                oppoent = teams[0]
            opp_name = [opponent for i in range(len(player_names))]
            df_opp_name = pd.DataFrame(opp_name)
            
            #Team ID
            team_name = [teams[team] for i in range(len(player_names))]
            df_team_name = pd.DataFrame(team_name)

            #Consolidating the Data:
            stats_tmp = pd.concat([df_game_id,df_team_name,df_opp_name, df_player_names,df_starters,df_team_stats_basic,df_team_stats_adv],axis=1)
            stats.append(stats_tmp)
        pbar.update(1)
    
    output_raw = pd.concat(stats)

Scraping Progress: 100%|█████████████████████████████████████████████████████████████| 399/399 [04:16<00:00,  1.55it/s]


## Data Post-Processing:

In [17]:
output = output_raw
#Add a header to the Data
header = ['Game_ID','Team','Opponent','Player','Starter','Min','FG','FGA','FG%','3P','3PA','3P%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','P/M',
          'del','TS%','eFG%','3PAr', 'FTr','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%','ORtg','DRtg','BPM']
output.columns = header

#Index Column - Reset IDs
output = output.reset_index(drop=True)

#Player Column - Replace any special characters with the 26 basic alphabetic characters.
output['Player'] = output['Player'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

#MP - Convert to decimal
for row in range(len(output['Min'])):
    if ":" in output['Min'][row]:
        output['Min'][row] = round(float(output['Min'][row].split(':')[0]+str(int(output['Min'][row].split(':')[1])/60)[1:]),2)
    else:
        output['Min'][row] = 0

#Delete redundant Minutes Played (del) column:
output = output.drop(['del'],axis=1)

#Convert statistical columns to float type in order to calculate fantasy points
output['PTS'] = pd.to_numeric(output['PTS'],downcast='float')
output['3P'] = pd.to_numeric(output['3P'],downcast='float')
output['TRB'] = pd.to_numeric(output['TRB'],downcast='float')
output['AST'] = pd.to_numeric(output['AST'],downcast='float')
output['STL'] = pd.to_numeric(output['STL'],downcast='float')
output['BLK'] = pd.to_numeric(output['BLK'],downcast='float')
output['TOV'] = pd.to_numeric(output['TOV'],downcast='float')

#Dealing with Double Doubles/Triple Doubles 
isTen = lambda x:int(x>=10)
countTens = lambda row: numpy.clip(isTen(row['PTS']) + isTen(row['TRB']) + isTen(row['AST']) + isTen(row['STL']) + isTen(row['BLK']),1,3)-1
output['DDBonus'] = output.apply(countTens,axis=1)

#Calculating Fantasy Points
output['Fp'] = 1*output['PTS'] + 0.5*output['3P'] + 1.25*output['TRB'] + 1.5*output['AST'] + 2*output['STL'] + 2*output['BLK'] - 0.5*output['TOV'] + 1.5*output['DDBonus']

#Calculating Fantasy Points per Minute
FpMin = []
for row in range(len(output['Min'])):
    if output['Min'][row] == 0:
        FpMin.append(0)
    else:
        FpMin.append(round(output['Fp'][row] / output['Min'][row],3))
output['Fp/Min'] = FpMin
        
#Replace any blank cells with 0
output = output.fillna(0)
output = output.replace('',0)

If preferred, run the kernel below to create a temporary .csv file in Binder

In [18]:
output.to_csv('2020stats.csv')

## Autoexport to MySQL Database

In [24]:
!pip install config
!pip install mysql
!pip install mysql.connector
!pip install sqlalchemy

You should consider upgrading via the 'c:\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python37\python.exe -m pip install --upgrade pip' command.


Using legacy 'setup.py install' for mysql.connector, since package 'wheel' is not installed.
    Running setup.py install for mysql.connector: started
    Running setup.py install for mysql.connector: finished with status 'done'


You should consider upgrading via the 'c:\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python37\python.exe -m pip install --upgrade pip' command.


In [26]:
import config
import mysql.connector 
from mysql.connector import errorcode
from sqlalchemy import create_engine

cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.passwd)
print(cnx)

cursor = cnx.cursor()
#insert Database Name
db_name = 'tjprzzy-nba-stats'

In [28]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

Database tjprzzy-nba-stats does not exists.
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '-nba-stats' at line 1


## Autoexport to Google Sheets

In order to access Google Sheets, we need credentials access.

Now the tricky part, getting credentials access. In order to use Python we need to get some form of key from Google. This key is mainly for security purposes and will be in the format of a JSON file.

In [11]:
#!pip install gspread
#!pip install gspread_dataframe

import gspread
from gspread_dataframe import set_with_dataframe

gc = gspread.oauth()
sh = gc.open_by_key('1ZmvBC3wiXHROO-I2aovyW2zF9t_z-gan4oDqRP0wSHE')
worksheet = sh.get_worksheet(0)

range_of_cells = worksheet.range('A1:AN50000') #-> Select the range you want to clear
for cell in range_of_cells:
    cell.value = ''
set_with_dataframe(worksheet, output)